## Trade scanner for VIX and consecutive weeks SPY closed lower

This program can be executed daily (after market open hours) as an easy way to determine if trades should be placed. It checks if the VIX value from the day before is below the 21-day exponential moving average of VIX, and also if VIX is increasing. If both conditions are met, the prompt prints a message indicating to the user to open trade.
- OpentradeTT - a more stringent rule which requires that both conditions are True on the same day.
- OpentradeFTT - a less stringent rule which requires that VIX is increasing, but if VIX is less than VIX-21D EMA the day before, open trade == True.

The program also checks the number of consecutive wks SPY has closed lower, for an overall indicator of the health of the stock market.

In [1]:
import pandas as pd            
import pandas_datareader.data as web
import datetime

In [2]:
import pandas_datareader.data as web

all_data = {ticker: web.get_data_yahoo(ticker)       #dictionary comprehension
           for ticker in ["^VIX","SPY"]}
# putting ["^VIX","SPY"] results in error during 315am to 915am ET due to uneven length of columns
# print(all_data)

try:
    vix_df0 = pd.DataFrame({ticker: data ['Adj Close']
               for ticker, data in all_data.items()})
except:
    print("Error encountered during copy from Yahoo Finance, should self resolve during market hours. putting [^VIX,SPY] results in error during 315am to 915am ET due to uneven length of columns")

vix_df = vix_df0.fillna(method="ffill")                 


In [3]:
vix_df = vix_df.loc[vix_df.index > '2021-12-31']
vix_df0 = vix_df0.loc[vix_df0.index > '2021-12-31']


In [4]:
vix_df.index = pd.to_datetime(vix_df.index)


vix_df['VIX_21d'] = vix_df['^VIX'].ewm(alpha=2/22, adjust=False).mean()        #calculates 21 day EMA from VIX value
vix_df = vix_df.fillna(method='bfill')                 
vix_df['^VIX-1d'] = vix_df['^VIX']
vix_df['^VIX-1d'] = vix_df['^VIX-1d'].shift(-1)                  #delete 1 cell and shift up once to compare against itself



date_index = pd.date_range(start=min(vix_df.index), end=max(vix_df.index), freq='D')
vix_df=vix_df.reindex(date_index, fill_value=0)

display(vix_df)


,^VIX,SPY,VIX_21d,^VIX-1d
2022-01-03,16.600000,476.230530,16.600000,16.910000
2022-01-04,16.910000,476.071014,16.628182,19.730000
2022-01-05,19.730000,466.929443,16.910166,19.610001
2022-01-06,19.610001,466.490784,17.155605,18.760000
2022-01-07,18.760000,464.646515,17.301459,19.400000
...,...,...,...,...
2022-05-30,0.000000,0.000000,0.000000,0.000000
2022-05-31,26.190001,412.929993,28.331234,25.690001
2022-06-01,25.690001,409.589996,28.091122,24.719999
2022-06-02,24.719999,417.390015,27.784657,25.450001


In [5]:
vix_df.tail(15)

,^VIX,SPY,VIX_21d,^VIX-1d
2022-05-20,29.430000,389.630005,29.026399,28.480000
2022-05-21,0.000000,0.000000,0.000000,0.000000
2022-05-22,0.000000,0.000000,0.000000,0.000000
2022-05-23,28.480000,396.920013,28.976726,29.450001
2022-05-24,29.450001,393.890015,29.019751,28.370001
2022-05-25,28.370001,397.369995,28.960683,27.500000
2022-05-26,27.500000,405.309998,28.827894,25.719999
2022-05-27,25.719999,415.260010,28.545358,26.190001
2022-05-28,0.000000,0.000000,0.000000,0.000000
2022-05-29,0.000000,0.000000,0.000000,0.000000


In [6]:
# Returns today's VIX values
tday = pd.Timestamp('today').normalize()
tday = str(tday)

In [7]:
vix_df['lt_21d'] = vix_df['VIX_21d'] > vix_df['^VIX']           #lt_21d checks if the current VIX is below the 21d EMA

vix_df['vix_inc'] = vix_df['^VIX-1d'] > vix_df['^VIX']         #vix_inc checks if vix is increasing, when compared to the VIX value the day before
vix_df['vix_inc'] = vix_df['vix_inc'].shift(1)

#open trade if both lt21d and vix_inc are True
vix_df['open_tradeTT'] = (vix_df['lt_21d'] & vix_df['vix_inc'])   

#open trade if lt21d is F and vix_inc is T and lt21d(previous day) is T --> for cases where VIX spikes suddenly making open_tradeTT false
vix_df['open_tradeFTT'] = (vix_df['lt_21d'] == False) & (vix_df['vix_inc'] == True) & (vix_df['lt_21d'].shift(1) == True)

# combine these 2 conditions
vix_df['open_trade'] = vix_df['open_tradeTT'] | vix_df['open_tradeFTT']

print('\nInstances where open trade = True')
display(vix_df.loc[vix_df['open_trade']==True])



Instances where open trade = True


,^VIX,SPY,VIX_21d,^VIX-1d,lt_21d,vix_inc,open_tradeTT,open_tradeFTT,open_trade
2022-02-02,22.090000,455.933594,23.761084,24.350000,True,True,True,False,True
2022-02-03,24.350000,445.216888,23.814621,23.219999,False,True,False,True,True
2022-02-10,23.910000,447.928467,23.224196,27.360001,False,True,False,True,True
2022-03-23,23.570000,443.799988,27.499687,21.670000,True,True,True,False,True
2022-03-30,19.330000,458.700012,24.636117,20.559999,True,True,True,False,True
2022-03-31,20.559999,451.640015,24.265560,19.629999,True,True,True,False,True
2022-04-05,21.030001,451.029999,23.152435,22.100000,True,True,True,False,True
2022-04-06,22.100000,446.519989,23.056759,21.549999,True,True,True,False,True
2022-04-09,0.000000,0.000000,0.000000,0.000000,False,True,False,True,True
2022-04-14,22.700001,437.790009,22.899386,22.170000,True,True,True,False,True


### Consecutive weeks that SPY dropped scanner

In [8]:
df = web.DataReader('SPY', 'yahoo', start='2022-01-01')
df['SPY'] = df['Close']
df = df.drop(['High','Low','Open','Volume','Adj Close', 'Close'], axis=1)

date_index = pd.date_range(start=min(df.index), end=max(df.index), freq='D')
df = df.reindex(date_index, method='nearest')
df['date1'] = df.index
df['day'] = df.date1.dt.dayofweek

df = df.loc[df.day == 4]               # take day = 0 or Monday as the start of the week
df['SPY-1wk'] = df['SPY'].shift(1)     #'SPY-1wk' or SPY minus 1 wk is the price of SPY 1 wk earlier
df['SPY-2wk'] = df['SPY'].shift(2)
df['SPY-3wk'] = df['SPY'].shift(3)
df['SPY-4wk'] = df['SPY'].shift(4)
df['SPY-5wk'] = df['SPY'].shift(5)
df['SPY+1wk'] = df['SPY'].shift(-1)

#checks for consecutive SPY declines

df['0<-1'] = df['SPY'] < df['SPY-1wk']
df['-1<-2'] = df['SPY-1wk'] < df['SPY-2wk']
df['-2<-3'] = df['SPY-2wk'] < df['SPY-3wk']
df['-3<-4'] = df['SPY-3wk'] < df['SPY-4wk']
df['-4<-5'] = df['SPY-4wk'] < df['SPY-5wk']
df['2consec_drops'] = (df['0<-1']) & (df['-1<-2'])
df['3consec_drops'] = (df['0<-1']) & (df['-1<-2']) & (df['-2<-3'])
df['4consec_drops'] = (df['0<-1']) & (df['-1<-2']) & (df['-2<-3']) & (df['-3<-4'])
df['5consec_drops'] = (df['0<-1']) & (df['-1<-2']) & (df['-2<-3']) & (df['-3<-4']) & (df['-4<-5'])

# display(df.tail(15))


## Consecutive wk drops in SPY, calculated for this week

In [9]:
df = df[['2consec_drops','3consec_drops','4consec_drops','5consec_drops']]
print('\nResults for this week')
df.tail(1)


Results for this week


,2consec_drops,3consec_drops,4consec_drops,5consec_drops
2022-05-30,False,False,False,False


## Describe VIX since 1 Jan 2022

In [10]:
display(vix_df0['^VIX'].describe())
# display(vix_df0)

count    106.000000
mean      25.988774
std        4.794323
min       16.600000
25%       21.992499
50%       25.710000
75%       30.110000
max       36.450001
Name: ^VIX, dtype: float64

## Describe VIX since given date

In [11]:
date = '2022-05-01'
vix_df1 = vix_df0.loc[vix_df0.index > date]
display(vix_df1['^VIX'].describe())
# display(vix_df1)

count    24.000000
mean     28.925834
std       2.782365
min      24.719999
25%      26.167500
50%      29.060000
75%      31.020000
max      34.750000
Name: ^VIX, dtype: float64

## Should I open a put trade cos VIX is below 21d EMA & VIX is inc?

In [12]:
try:
    if vix_df.loc[tday,'open_trade'] == True:
        print("<<<< Yes, open Trade today! >>>>")
    else:
        print("<<<< No, keep on waiting >>>> ")
        
except:
    print("KEY ERROR because today's date,",tday,"is not found in database yet!")
    
print("\nToday's VIX data")
display(vix_df.tail(4))



<<<< Yes, open Trade today! >>>>

Today's VIX data


,^VIX,SPY,VIX_21d,^VIX-1d,lt_21d,vix_inc,open_tradeTT,open_tradeFTT,open_trade
2022-05-31,26.190001,412.929993,28.331234,25.690001,True,False,False,False,False
2022-06-01,25.690001,409.589996,28.091122,24.719999,True,False,False,False,False
2022-06-02,24.719999,417.390015,27.784657,25.450001,True,False,False,False,False
2022-06-03,25.450001,417.390015,27.572415,NaN,True,True,True,False,True
